In [10]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [11]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# GLOBALS
e = np.e

# HYPERPARAMETERS
epochs = 300
alpha = .2 # learning rate

data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
data = np.array(data)
rows, features = data.shape # saves rows and columns of dataset
np.random.shuffle(data)

data_dev = data[0:1000].T # data used for validation; the first 1k from MNST, transposed into columns
Y_dev = data_dev[0] # Y (ground truth) is the new first row of transposed subset, which contains all labels of the subset
X_dev = data_dev[1:features] # everything that's not a label (so pixel values in this case)
X_dev = X_dev / 255. #normalize image

#data_train = data[200:rows].T # training dataset
data_train = data[1000:rows].T # training dataset
Y_train = data_train[0] # all labels of train
X_train = data_train[1:features] # all non-labels, each row is a pixel location, each column is one image
X_train = X_train / 255. #normalize image

def make_weights(a, b): 
    return np.random.rand(a, b) * 0.01, np.zeros(b)

def initial_params():
    # inputs -> hidden layer 1
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5 # initial weights = random between -.5 and .5, and u make 10x784 matrix
    #cus of the amount of pixels in the first image. for this example youd do 100 actually, but 784 for
    # the mnit stuff
    # hidden layer 2
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    # fully connected layer 3, aka output layer
    W3 = np.random.rand(10, 10) - 0.5
    b3 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2, W3, b3

def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1 # for each individual Z_n = W_n*A_(n-1) + b_n, and this is like the matrix/
    # vectors/fancy linear algebra version of doing that (instead of loops
    # bc computers <3 matrices or whatever)
    
    # this is the first forward propagation, so X (the flattened image vector) represents the A_(n-1),
    # aka activations of the first layer
    
    A1 = ReLU(Z1) # Z calculation passed through neuron activation function. first activation/full estimate

    Z2 = W2.dot(A1) + b2 # second layer's calculation w/ weights and biases, based on A1 estimate. 
    A2 = ReLU(Z2) # activations of the 2nd layer's weights and biases calculations

    Z3 = W3.dot(A2) + b3 # thrd layer's calculation w/ weights and biases, based on A2 estimate. 
    A3 = softmax(Z3) # activations of the 3rd layer's weights and biases calculations

    return Z1, A1, Z2, A2, Z3, A3 # give back the activation values (A) and the stuff resulting from weights and biases (Z)
    
    
def Z (a_prev: float, w: float, b: float):
    # given previous value, weight, and bias, returns z estimate
    return np.dot(a_prev,w)+b
    
def ReLU(z):
    # FUNC: activation/deactivation of neurons by setting them to
    # if z > 0:
    #     return z
    # else:
    #     return 0
    # lil graveyard for when we thought we would use loops insted of matrices o7
    return np.maximum(0, z) # returns np array w zero if z val <= 0, and z otherwise

def deriv_relu(z):
    # bro this is so dumb idek but like whatever
    # if z > 0:
    #     return 1
    # else:
    #     return 0
    # relu = ReLU(z) # relu func
    # relu[relu > 0] = 1 # sets all relu'd zs > 0 to 1, and keeps the rest 
    # return relu
    return z > 0  

# def softmax(Z):
#     Z_probs = np.exp(Z) / sum(np.exp(Z)) 
#     return Z_probs
# def softmax(Z):
#     expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))  # numerical stability
#     return expZ / np.sum(expZ, axis=0, keepdims=True)

def softmax(Z):
    # Z is (10, m)
    Z_exp = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return Z_exp / np.sum(Z_exp, axis=0, keepdims=True)

# backwards propagation: estimates -> error -> how much weights/biases contributed to error -> adjustments
    
def backpropagation(Z1, A1, Z2, A2, Z3, A3, W2, W3, X, Y):
    m = X.shape[1]
    # goes backwards through layers to find their contribution to loss

    one_hot_Y = one_hot(Y) # get the one hot stuff for Y; so the [0 0 1]-esque shaped thing, representing what the
    # final probabilities ideally should be, if the nn was fully accurate for this example, in order to compare it
    
    dZ3 = A3 - one_hot_Y # how wrong the estimatinos are
    dW3 = 1/m * dZ3 @ A2.T 
    # you can derive thisss using skibidi rizz magci hahahahahahahahahahahahahahah
    # sorry, dunno what happened, i meant to say that the avg change in loss in layer 3 with respect to weight
    # = the average of the matrixly multiplied (??) gradients of the loss w/ respect to weight. that was very circular
    # the weight comes from the W within the Z because like W is in the Z. im getting tired
    db3 = 1/m * np.sum(dZ3) # the average of the absolute error
    # not mean squared error btw

    # holy cow FINALLY WE'RE ON LAYER 2. chris u have so much faith in us. idk if its desrverd. but ty ig
    dZ2 = W3.T @ dZ3 * deriv_relu(Z2)
    dW2 = 1/m * dZ2 @ A1.T
    db2 = 1/m * np.sum(dZ2)

    # layer 1!!!!!! youve seen the annotations
    dZ1 = W2.T @ dZ2 * deriv_relu(Z1)
    dW1 = 1/m * dZ1 @ X.T
    # rhis time, its using X.T, because that's what A0 is
    db1 = 1/m * np.sum(dZ1)

    return dW3, db3, dW2, db2, dW1, db1 # returns all partial derivatives of weights and biases

def update_parameters(W1, b1, W2, b2, W3, b3, dW3, db3, dW2, db2, dW1, db1, alpha):
    # FUNC: updates weights and biases based off of their contributions to loss and the learning rate (alpha)
    W1 = W1 - alpha * dW1
    W2 = W2 - alpha * dW2
    W3 = W3 - alpha * dW3

    b1 = b1 - alpha * db1
    b2 = b2 - alpha * db2
    b3 = b3 - alpha * db3
    
    return W1, b1, W2, b2, W3, b3

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2, W3, b3 = initial_params()

    for i in range (iterations): # runs forward prop, back prop, update params for # iterations told
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X) 
        dW3, db3, dW2, db2, dW1, db1 = backpropagation(Z1, A1, Z2, A2, Z3, A3, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_parameters(W1, b1, W2, b2, W3, b3, dW3, db3, dW2, db2, dW1, db1, alpha)        
        #see function comments
    
        if i % 10 == 0: # prints stats for every tenth iteration
            print(f"Epoch: {i}") # prints iteration number! rahh
            print(f"Accuracy: {get_accuracy(get_predictions(A3), Y)}") # prints accuracy
    print(f"Total Epochs: {i}") # prints iteration number! rahh
    print(f"Final Accuracy: {get_accuracy(get_predictions(A3), Y)}") # prints accuracy
    return W1, b1, W2, b2, W3, b3

def get_predictions(A3):
    return np.argmax(A3, 0) # get biggest final activation, if theres nothing just say 0

def get_accuracy(predictions, Y):
    # get accuracy of predictions
    return np.sum(predictions == Y) / Y.size # how oftem predictions are right divded by the amt samples tested

def get_losses(estimates: np.array, ground_truths: np.array):
    losses = []

    for estimate, truth, in zip(estimates, ground_truths):
        losses.append(estimate - truth)
        
    return losses

def one_hot(Y):
    # make a matrix where each row is 0 if it is not the label number, and 1 if it is.
    one_hot_y = np.zeros((Y.size, Y.max() + 1)) # empty numpy array of zeroes with a column for each digit possibility
    # or 10, cuz (0, 1, 2, ..., 9)
    one_hot_y[np.arange(Y.size), Y] = 1 # for each image/row (Y.size), set column corresponding to the digit value (index = Y) to 1 (true)
    one_hot_y = one_hot_y.T # transposes; makes each column an example, instead of digit
    return one_hot_y

In [13]:
W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, epochs, alpha)

Epoch: 0
Accuracy: 0.09102439024390244
Epoch: 10
Accuracy: 0.15682926829268293
Epoch: 20
Accuracy: 0.2268048780487805
Epoch: 30
Accuracy: 0.36817073170731707
Epoch: 40
Accuracy: 0.4517317073170732
Epoch: 50
Accuracy: 0.44582926829268293
Epoch: 60
Accuracy: 0.5107317073170732
Epoch: 70
Accuracy: 0.5907317073170731
Epoch: 80
Accuracy: 0.6115121951219512
Epoch: 90
Accuracy: 0.6480243902439025
Epoch: 100
Accuracy: 0.6751707317073171
Epoch: 110
Accuracy: 0.6990243902439024
Epoch: 120
Accuracy: 0.7136341463414634
Epoch: 130
Accuracy: 0.7269512195121951
Epoch: 140
Accuracy: 0.7426585365853658
Epoch: 150
Accuracy: 0.7534146341463415
Epoch: 160
Accuracy: 0.763
Epoch: 170
Accuracy: 0.7716341463414634
Epoch: 180
Accuracy: 0.783
Epoch: 190
Accuracy: 0.7964634146341464
Epoch: 200
Accuracy: 0.7917804878048781
Epoch: 210
Accuracy: 0.8065365853658537
Epoch: 220
Accuracy: 0.8125121951219512
Epoch: 230
Accuracy: 0.8141219512195121
Epoch: 240
Accuracy: 0.8225121951219512
Epoch: 250
Accuracy: 0.8257317073

In [14]:
print("Test Accuracy:")
Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X_dev) #forward propagate through test dataset with finalized weights
predictions = get_predictions(A3) # get predictions
print("Epoch: ", epochs, "Accuracy: ", get_accuracy(predictions, Y_dev)) #print the accuracy of the predictions

Test Accuracy:
Epoch:  300 Accuracy:  0.807
